# Задачки для стажера

### 1. Какие шаги ты бы предпринял, если бы пользователь сказал, что API возвращает ему ошибку 500?

#### Решение
1. Спросил бы у пользователя: какой метод API вызывался, в какой отрезок время был сделан запрос и его request-id;
2. В системе логирования сделал бы запрос со следующими значениями: временной отрезок и request-id пользователя;
3. Просмотрел бы последовательность действий пользователя до появления ошибки и изучил бы лог ошибки;
4. Дальше если это баг в кодовой базе, то написал бы отчет о баге и если в моих силах это исправить, то сделал бы pull request на исправление ошибки
5. Если я понятия не имею из-за чего произошла ошибка, то обратился бы к куратору(наставнику) или коллегам за помощью

### 2. Какие ты видишь проблемы в следующем фрагменте кода ? Как его следует исправить ? Исправь ошибку и перепиши код ниже с использованием типизации.
```python
def create_handlers(callback):
    handlers = []
    for step in range(5):
        # добавляем обработчики для каждого шага (от 0 до 4)
        handlers.append(lambda: callback(step))
    return handlers

def execute_handlers(handlers):
    # запускаем добавленные обработчики (шаги от 0 до 4)
    for handler in handlers:
        handler()
```

#### Решение
Когда в цикле создается обработчик в него передается не значение локальной переменной step, а ссылка, из-за этого при выполнении обработчиков используется последние значение переменной step.
Это можно исправить применив функцию partial из модуля functools, он оборачивает функцию с аргументами и при вызове явно передает аргументы в функцию.
- https://stackoverflow.com/a/34021333
- https://docs.python.org/3.4/faq/programming.html#why-do-lambdas-defined-in-a-loop-with-different-values-all-return-the-same-result

In [97]:
from typing import Callable, List
from functools import partial


def create_handlers(callback: Callable) -> List[Callable]:
    handlers = []
    for step in range(5):
        # добавляем обработчики для каждого шага (от 0 до 4)
        handlers.append(partial(callback, step))
    return handlers


def execute_handlers(handlers: List[Callable]) -> None:
    # запускаем добавленные обработчики (шаги от 0 до 4)
    for handler in handlers:
        handler()


handlers = create_handlers(lambda x: print(x, id(x)))
execute_handlers(handlers)


0 140715437135600
1 140715437135632
2 140715437135664
3 140715437135696
4 140715437135728


### 3. Сколько HTML-тегов в коде главной страницы сайта greenatom.ru ? Сколько из них содержит атрибуты ? Напишите скрипт на Python, который выводит ответы на вопросы выше.

In [98]:
from bs4 import BeautifulSoup
import requests

url = r"https://greenatom.ru/"
headers = {"User-agent": "Mozilla/5.0 (Windows; U; Windows NT 6.1) AppleWebKit/534.5.7 (KHTML, like Gecko) Version/5.1 Safari/534.5.7"}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")

tags_count = len(soup.find_all())
tags_with_attr_count = len(soup.find_all(lambda tag: tag.attrs))

print(f"Сайта: {url}")
print(f"Количество HTML-тегов на странице: {tags_count}")
print(f"Количество HTML-тегов с атрибутами на странице: {tags_with_attr_count}")

Сайта: https://greenatom.ru/
Количество тегов на странице: 774
Количество тегов с атрибутами на странице: 478


### 4. Напиши функцию на Python, которая возвращает текущий публичный IP-адрес компьютера (например, с использованием сервиса ifconfig.me).

In [ ]:
import requests

url = r"https://ifconfig.me/ip"
headers = {"User-agent": "Mozilla/5.0 (Windows; U; Windows NT 6.1) AppleWebKit/534.5.7 (KHTML, like Gecko) Version/5.1 Safari/534.5.7"}
response = requests.get(url, headers=headers)

print(f"Ваш IP: {response.text}")

### 5. Напиши функцию на Python, выполняющую сравнение версий
Условия:
- Return -1 if version A is older than version B
- Return 0 if versions A and B are equivalent
- Return 1 if version A is newer than version B
- Each subsection is supposed to be interpreted as a number, therefore 1.10 > 1.1.

In [3]:
def compare_versions(version_a: str, version_b: str) -> int:
    _version_a = version_a.split(".")
    _version_b = version_b.split(".")

    try:
        _version_a = tuple(map(int, _version_a))
        _version_b = tuple(map(int, _version_b))
    except ValueError:
        raise ValueError(
            "Each subsection is supposed to be interpreted as a number.\n"
            f"Present this values: {version_a=}; {version_b=};"
        )

    if _version_a < _version_b:
        return -1
    
    elif _version_a > _version_b:
        return 1

    elif _version_a == _version_b:
        return 0


assert compare_versions("3.10", "3.7.3") == 1
assert compare_versions("3.7", "3.7.1") == -1
assert compare_versions("3.7", "3.7") == 0
try:
    compare_versions("3.7", "3.7a")
except ValueError as ex:
    print(ex)

Each subsection is supposed to be interpreted as a number.
Present this values: version_a='3.7'; version_b='3.7a';
